# SPARC-P PubApps Deployment (Pixel Streaming)

This notebook deploys SPARC-P using server-side Unity rendering for thin clients on a single L4 GPU (24GB).

## Architectural Changes
- Remove Audio2Face
- Run Unity Linux Server container with Render Streaming
- Add Node signaling container for WebRTC negotiation
- Keep FastAPI + LLM + Riva as local services

In [ ]:
import os
import textwrap
from pathlib import Path

PROJECT = os.environ.get("SPARC_PUBAPPS_PROJECT", "SPARCP")
PUBAPPS_ROOT = Path(f"/pubapps/{PROJECT}")
MODEL_DIR = PUBAPPS_ROOT / "models"
RIVA_MODEL_DIR = PUBAPPS_ROOT / "riva_models"
QUADLET_DIR = Path.home() / ".config/containers/systemd"

VRAM_PLAN = {
    "llm_vllm_4bit": "~13GB",
    "unity_render": "~3.5GB",
    "riva_embedded": "~3GB",
    "buffer": "~2-3GB"
}

print(f"Project: {PROJECT}")
print(f"Root: {PUBAPPS_ROOT}")
print(f"VRAM plan: {VRAM_PLAN}")

In [ ]:
EXECUTE = False

def run(cmd: str):
    print(f"$ {cmd}")
    if not EXECUTE:
        print("(dry-run) command not executed\n")
        return
    os.system(cmd)

## 1) Prepare directories and base resources

In [ ]:
run(f"mkdir -p {PUBAPPS_ROOT} {MODEL_DIR} {RIVA_MODEL_DIR} {PUBAPPS_ROOT / 'logs'}")
run(f"mkdir -p {QUADLET_DIR}")

## 2) Build/pull required images

In [ ]:
build_cmds = textwrap.dedent("""
podman build -f Dockerfile.mas -t sparc/llm-backend:latest .
podman build -f Dockerfile.unity-server -t sparc/unity-server:latest .
podman build -f Dockerfile.signaling -t sparc/signaling-server:latest .
podman pull nvcr.io/nvidia/riva/riva-speech:2.16.0-server
""").strip()
print(build_cmds)

## 3) Write Podman Quadlet units

In [ ]:
pod_content = textwrap.dedent("""
[Unit]
Description=SPARC-P Avatar Pod (Pixel Streaming)
After=network-online.target

[Pod]
PodName=sparc-avatar
PublishPort=8000:8000
PublishPort=8080:8080
PublishPort=3478:3478/udp
PublishPort=49152-49200:49152-49200/udp
PublishPort=49152-49200:49152-49200/tcp

[Install]
WantedBy=default.target
""").strip()
(QUADLET_DIR / "avatar.pod").write_text(pod_content)
print("Wrote avatar.pod")

In [ ]:
riva_container = textwrap.dedent(f"""
[Unit]
Description=SPARC-P Riva Speech Server
After=avatar-pod.service
Requires=avatar-pod.service

[Container]
Pod=sparc-avatar.pod
Image=nvcr.io/nvidia/riva/riva-speech:2.16.0-server
ContainerName=riva-server
Volume={RIVA_MODEL_DIR}:/data:Z
Device=nvidia.com/gpu=all
Environment=NVIDIA_VISIBLE_DEVICES=all
Exec=/opt/riva/bin/riva_server --riva_model_repo=/data/models

[Service]
Restart=always
TimeoutStartSec=300

[Install]
WantedBy=default.target
""").strip()
(QUADLET_DIR / "riva-server.container").write_text(riva_container)
print("Wrote riva-server.container")

In [ ]:
backend_container = textwrap.dedent(f"""
[Unit]
Description=SPARC-P FastAPI + vLLM Backend
After=avatar-pod.service riva-server.service
Requires=avatar-pod.service riva-server.service

[Container]
Pod=sparc-avatar.pod
Image=sparc/llm-backend:latest
ContainerName=sparc-backend
Volume={MODEL_DIR}:{MODEL_DIR}:Z
Environment=MODEL_ID=gpt-oss-20b
Environment=QUANTIZATION=4bit
Environment=RIVA_SERVER=localhost:50051
Environment=SPARC_MODEL_BASE_PATH={MODEL_DIR}
Environment=SPARC_FIREBASE_CREDS={PUBAPPS_ROOT / 'config' / 'firebase-credentials.json'}
Device=nvidia.com/gpu=all
Exec=uvicorn main:app --host 0.0.0.0 --port 8000 --workers 1

[Service]
Restart=always
RestartSec=10

[Install]
WantedBy=default.target
""").strip()
(QUADLET_DIR / "sparc-backend.container").write_text(backend_container)
print("Wrote sparc-backend.container")

In [ ]:
signaling_container = textwrap.dedent("""
[Unit]
Description=SPARC-P Unity Render Streaming Signaling Server
After=avatar-pod.service
Requires=avatar-pod.service

[Container]
Pod=sparc-avatar.pod
Image=sparc/signaling-server:latest
ContainerName=signaling-server
Environment=HTTP_PORT=8080
Exec=node server.js --httpPort 8080

[Service]
Restart=always
RestartSec=5

[Install]
WantedBy=default.target
""").strip()
(QUADLET_DIR / "signaling-server.container").write_text(signaling_container)
print("Wrote signaling-server.container")

In [ ]:
unity_container = textwrap.dedent("""
[Unit]
Description=SPARC-P Unity Linux Server (Render Streaming)
After=avatar-pod.service signaling-server.service sparc-backend.service riva-server.service
Requires=avatar-pod.service signaling-server.service sparc-backend.service riva-server.service

[Container]
Pod=sparc-avatar.pod
Image=sparc/unity-server:latest
ContainerName=unity-server
Device=nvidia.com/gpu=all
Environment=SIGNALING_URL=ws://localhost:8080
Environment=BACKEND_URL=http://localhost:8000
Environment=RIVA_URL=localhost:50051
Exec=/app/SPARC-P.x86_64 -logFile /dev/stdout -batchmode -force-vulkan

[Service]
Restart=always
RestartSec=5

[Install]
WantedBy=default.target
""").strip()
(QUADLET_DIR / "unity-server.container").write_text(unity_container)
print("Wrote unity-server.container")

In [ ]:
enable_cmds = textwrap.dedent("""
systemctl --user daemon-reload
systemctl --user enable --now avatar-pod
systemctl --user enable --now riva-server
systemctl --user enable --now sparc-backend
systemctl --user enable --now signaling-server
systemctl --user enable --now unity-server
nvidia-smi
""").strip()
print(enable_cmds)